# 🌊 Udawalawe Reservoir Forecasting Model
## AI-Driven Probabilistic Forecasting for Water Management
---
**Objective:** Develop ML models to forecast inflow, storage level, and irrigation demand (1-14 day horizon)

**Models Implemented:**
- ARIMA (Time-Series Baseline)
- Random Forest / Gradient Boosting (ML)
- LSTM / GRU (Deep Learning)
- Quantile Regression (Probabilistic Outputs)

## 1. Setup & Dependencies

In [1]:
# Install required packages if not already installed
%pip install -q tensorflow statsmodels

# Core Libraries
import pandas as pd
import numpy as np
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')

# Preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ML Models
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Time Series
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

# Deep Learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

print('All libraries imported successfully!')
print(f'TensorFlow version: {tf.__version__}')

Note: you may need to restart the kernel to use updated packages.
All libraries imported successfully!
TensorFlow version: 2.20.0


## 1.5 Download CHIRPS + NASA POWER Data

This section downloads remotely-sensed meteorological and precipitation data:
- **CHIRPS**: Climate Hazards Group InfraRed Precipitation with Stations (via ClimateSERV API)
- **NASA POWER**: Prediction Of Worldwide Energy Resource (daily meteorology)
- **ET₀**: Reference Evapotranspiration (computed from meteorological data using Hargreaves method)

**Study Area**: Udawalawe Reservoir, Sri Lanka (~6.5°N, 80.75°E)

In [2]:
# Import the data downloader module
import sys
sys.path.insert(0, '.')

from data_downloader import (
    download_and_integrate, 
    CHIRPSDownloader, 
    NASAPOWERDownloader,
    DataIntegrator
)

print("✓ Data downloader module loaded successfully")

✓ Data downloader module loaded successfully


In [3]:
# Configuration: Udawalawe Reservoir coordinates
LAT = 6.5                              # Latitude (°N)
LON = 80.75                            # Longitude (°E)
ELEVATION = 380                        # Elevation (m above sea level)

# Date range for data download
# NOTE: Adjust these dates based on your hydrological data availability
START_DATE = "2020-01-01"
END_DATE = "2023-12-31"

print(f"Download Configuration:")
print(f"  Location: Udawalawe Reservoir ({LAT}°N, {LON}°E)")
print(f"  Elevation: {ELEVATION}m")
print(f"  Date Range: {START_DATE} to {END_DATE}")
print(f"  Duration: {(pd.to_datetime(END_DATE) - pd.to_datetime(START_DATE)).days} days")

Download Configuration:
  Location: Udawalawe Reservoir (6.5°N, 80.75°E)
  Elevation: 380m
  Date Range: 2020-01-01 to 2023-12-31
  Duration: 1460 days


In [4]:
# Download integrated meteorological and precipitation data
# use_synthetic_chirps=True uses realistic synthetic data when ClimateSERV API is unavailable
# Set to False to use live API calls (requires internet and may take longer)

print("\n" + "="*70)
print("DOWNLOADING METEOROLOGICAL DATA FROM REMOTE SOURCES")
print("="*70 + "\n")

# Download with synthetic CHIRPS for testing (fast, no API calls)
meteorological_df = download_and_integrate(
    lat=LAT,
    lon=LON,
    start_date=START_DATE,
    end_date=END_DATE,
    elevation=ELEVATION,
    use_synthetic_chirps=True  # Set to False to use live CHIRPS API
)

print(f"\n✓ Downloaded {len(meteorological_df)} meteorological records")
print(f"\nFirst 5 rows:")
print(meteorological_df.head())

print(f"\nData types and info:")
print(meteorological_df.dtypes)

2026-01-03 14:37:51,811 - INFO - ============================================================
2026-01-03 14:37:51,812 - INFO - STARTING INTEGRATED DATA DOWNLOAD
2026-01-03 14:37:51,813 - INFO - ============================================================
2026-01-03 14:37:51,815 - INFO - Downloading NASA POWER data (2020-01-01 to 2023-12-31)...



DOWNLOADING METEOROLOGICAL DATA FROM REMOTE SOURCES



2026-01-03 14:37:52,978 - ERROR - NASA POWER download failed: 404 Client Error: Not Found for url: https://power.larc.nasa.gov/api/v1/daily?start=20200101&end=20231231&latitude=6.5&longitude=80.75&parameters=T2M%2CT2M_MAX%2CT2M_MIN%2CRH2M%2CWS2M%2CALLSKY_SFC_SW_DWN&community=ag&format=json
2026-01-03 14:37:52,990 - INFO - Generating synthetic CHIRPS data (for testing)...
2026-01-03 14:37:53,029 - INFO - ✓ Generated 48 synthetic CHIRPS records
2026-01-03 14:37:53,031 - INFO - 
2026-01-03 14:37:53,032 - INFO - DATA SUMMARY
2026-01-03 14:37:53,033 - INFO - ============================================================
2026-01-03 14:37:53,035 - INFO - Date range: 2020-01-01 00:00:00 to 2023-12-01 00:00:00
2026-01-03 14:37:53,036 - INFO - Records: 48
2026-01-03 14:37:53,038 - INFO - 
Columns and Missing Values:
2026-01-03 14:37:53,040 - INFO -   precipitation_mm: 0.0% missing



✓ Downloaded 48 meteorological records

First 5 rows:
            precipitation_mm
date                        
2020-01-01          7.190419
2020-02-01         49.750254
2020-03-01         94.420505
2020-04-01        120.114964
2020-05-01        144.290669

Data types and info:
precipitation_mm    float64
dtype: object


## 2. Load & Explore Data

In [5]:
# Load your hydrological data
DATA_PATH = 'data/01. Hydrological Data 1994 to 2025.xlsx'

df = pd.read_excel(DATA_PATH, header=2)

# Clean column names
df.columns = [col.replace('\n', '_').replace(' ', '_') for col in df.columns]

print(f'Dataset Shape: {df.shape}')
print(f'\nColumns: {list(df.columns)}')
df.head()

Dataset Shape: (365, 16)

Columns: ['Date', 'Water_Level_(mMSL)', 'Water_Level_(ftMSL)', 'Total_Storage_(MCM)', 'Active_Storage_(MCM)', 'Gross_Storage_Percentage_(%)', 'Net_Percentage_(%)', 'Energy_(MWh)', 'LB_Main_Canal_(MCM)', 'RB_Main_Canal_(MCM)', 'Main_Canals:_LB+RB_(MCM)', 'Spillway_(MCM)', 'Evaporation_(mm)', 'Evaporation_(MCM)', 'Inflow_(MCM)', 'Rainfall_(mm)']


,Date,Water_Level_(mMSL),Water_Level_(ftMSL),Total_Storage_(MCM),Active_Storage_(MCM),Gross_Storage_Percentage_(%),Net_Percentage_(%),Energy_(MWh),LB_Main_Canal_(MCM),RB_Main_Canal_(MCM),Main_Canals:_LB+RB_(MCM),Spillway_(MCM),Evaporation_(mm),Evaporation_(MCM),Inflow_(MCM),Rainfall_(mm)
0,1994-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1994-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1994-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1994-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1994-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Data Quality Report
print('='*60)
print('DATA QUALITY REPORT')
print('='*60)

# Get date column (first column)
date_col = df.columns[0]
print(f'\nDate Range: {df[date_col].min()} to {df[date_col].max()}')
print(f'Total Records: {len(df)}')

print('\nMissing Values:')
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
print(pd.DataFrame({'Missing': missing, '%': missing_pct}))

DATA QUALITY REPORT

Date Range: 1994-01-01 00:00:00 to 1994-12-31 00:00:00
Total Records: 365

Missing Values:
                              Missing       %
Date                                0    0.00
Water_Level_(mMSL)                160   43.84
Water_Level_(ftMSL)               365  100.00
Total_Storage_(MCM)               365  100.00
Active_Storage_(MCM)              365  100.00
Gross_Storage_Percentage_(%)      365  100.00
Net_Percentage_(%)                365  100.00
Energy_(MWh)                      365  100.00
LB_Main_Canal_(MCM)               365  100.00
RB_Main_Canal_(MCM)               365  100.00
Main_Canals:_LB+RB_(MCM)          365  100.00
Spillway_(MCM)                    365  100.00
Evaporation_(mm)                  365  100.00
Evaporation_(MCM)                 365  100.00
Inflow_(MCM)                      160   43.84
Rainfall_(mm)                     160   43.84


## 3. Data Preprocessing

In [7]:
# Rename columns for easier access
df = df.rename(columns={df.columns[0]: 'date'})
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date').sort_index()

# Identify numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f'Numeric columns: {numeric_cols}')

Numeric columns: ['Water_Level_(mMSL)', 'Water_Level_(ftMSL)', 'Total_Storage_(MCM)', 'Active_Storage_(MCM)', 'Gross_Storage_Percentage_(%)', 'Net_Percentage_(%)', 'Energy_(MWh)', 'LB_Main_Canal_(MCM)', 'RB_Main_Canal_(MCM)', 'Main_Canals:_LB+RB_(MCM)', 'Spillway_(MCM)', 'Evaporation_(mm)', 'Evaporation_(MCM)', 'Inflow_(MCM)', 'Rainfall_(mm)']


In [8]:
# Impute missing values - focus on columns that have some data
print(f'Missing values before imputation: {df.isnull().sum().sum()}')
print(f'\\nColumns with all NaN (will be dropped):')
all_nan_cols = df.columns[df.isnull().all()].tolist()
print(all_nan_cols)

# Drop columns that are entirely NaN
df = df.dropna(axis=1, how='all')
print(f'\\nShape after dropping all-NaN columns: {df.shape}')

# Update numeric_cols to exclude the columns we just dropped
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f'Numeric columns after cleanup: {numeric_cols[:5]}...')  # Show first 5

# Impute remaining missing values in numeric columns
for col in numeric_cols:
    missing_pct = df[col].isnull().sum() / len(df) * 100
    if missing_pct > 0:
        # For columns with some data, interpolate
        df[col] = df[col].interpolate(method='linear', limit=7)
        # Forward fill then backward fill
        df[col] = df[col].bfill().ffill()
        # Fill any remaining with mean
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].mean())

print(f'\\nMissing after imputation: {df.isnull().sum().sum()}')

Missing values before imputation: 4860
\nColumns with all NaN (will be dropped):
['Water_Level_(ftMSL)', 'Total_Storage_(MCM)', 'Active_Storage_(MCM)', 'Gross_Storage_Percentage_(%)', 'Net_Percentage_(%)', 'Energy_(MWh)', 'LB_Main_Canal_(MCM)', 'RB_Main_Canal_(MCM)', 'Main_Canals:_LB+RB_(MCM)', 'Spillway_(MCM)', 'Evaporation_(mm)', 'Evaporation_(MCM)']
\nShape after dropping all-NaN columns: (365, 3)
Numeric columns after cleanup: ['Water_Level_(mMSL)', 'Inflow_(MCM)', 'Rainfall_(mm)']...
\nMissing after imputation: 0


## 4. Feature Engineering

In [9]:
# Set target column - adjust based on your data
# Options: inflow, total_storage, water_level, etc.
TARGET = numeric_cols[0]  # Use first numeric column as default
print(f'Target variable: {TARGET}')

# Create temporal features
df['day_of_year'] = df.index.dayofyear
df['month'] = df.index.month
df['week'] = df.index.isocalendar().week.astype(int)

# Cyclical encoding
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

# Lag features - adjust these based on your data availability
# For shorter datasets, use fewer/shorter lags
lag_days = [1, 3, 7]  # Reduced from [1, 3, 7, 14] to avoid excessive NaN rows
for lag in lag_days:
    df[f'{TARGET}_lag_{lag}'] = df[TARGET].shift(lag)

# Rolling statistics - with min_periods to avoid excessive NaN
df[f'{TARGET}_roll_mean_7'] = df[TARGET].rolling(window=7, min_periods=1).mean()
df[f'{TARGET}_roll_std_7'] = df[TARGET].rolling(window=7, min_periods=1).std()

print(f'Features created. Shape: {df.shape}')
print(f'Missing values in df:')
print(df.isnull().sum())

Target variable: Water_Level_(mMSL)
Features created. Shape: (365, 13)
Missing values in df:
Water_Level_(mMSL)                0
Inflow_(MCM)                      0
Rainfall_(mm)                     0
day_of_year                       0
month                             0
week                              0
month_sin                         0
month_cos                         0
Water_Level_(mMSL)_lag_1          1
Water_Level_(mMSL)_lag_3          3
Water_Level_(mMSL)_lag_7          7
Water_Level_(mMSL)_roll_mean_7    0
Water_Level_(mMSL)_roll_std_7     1
dtype: int64


## 5. Train-Test Split

In [10]:
# Drop NaN rows from lag features and rolling statistics
# Keep track of original size for diagnostics
original_size = len(df)
df_model = df.dropna()
dropped_rows = original_size - len(df_model)
print(f'Original dataset: {original_size} rows')
print(f'After dropping NaN: {len(df_model)} rows (dropped {dropped_rows})')

# Ensure we have enough data
if len(df_model) == 0:
    raise ValueError("All data was dropped. Check if dataset is large enough and lag features aren't too aggressive.")

# Time-based split (80/20)
train_size = int(len(df_model) * 0.8)
train_df = df_model.iloc[:train_size]
test_df = df_model.iloc[train_size:]

print(f'\nTrain-Test Split:')
print(f'  Training: {len(train_df)} samples')
print(f'  Testing: {len(test_df)} samples')

# Separate features and target
feature_cols = [c for c in df_model.columns if c != TARGET]

X_train, y_train = train_df[feature_cols], train_df[TARGET]
X_test, y_test = test_df[feature_cols], test_df[TARGET]

# Final validation
if len(X_train) == 0:
    raise ValueError("Training set is empty. Consider reducing lag features or using a larger dataset.")

print(f'\nFeature matrices:')
print(f'  X_train shape: {X_train.shape}')
print(f'  X_test shape: {X_test.shape}')
print(f'  Features: {len(feature_cols)}')

Original dataset: 365 rows
After dropping NaN: 358 rows (dropped 7)

Train-Test Split:
  Training: 286 samples
  Testing: 72 samples

Feature matrices:
  X_train shape: (286, 12)
  X_test shape: (72, 12)
  Features: 12


In [11]:
# Scale data
print('Scaling data...')
print(f'  X_train: {X_train.shape}, y_train: {y_train.shape}')
print(f'  X_test: {X_test.shape}, y_test: {y_test.shape}')

if len(X_train) == 0:
    raise ValueError("Cannot scale empty training data. Check previous cells for data loading issues.")

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

print('✓ Data scaled successfully!')
print(f'  X_train_scaled shape: {X_train_scaled.shape}')
print(f'  X_test_scaled shape: {X_test_scaled.shape}')

Scaling data...
  X_train: (286, 12), y_train: (286,)
  X_test: (72, 12), y_test: (72,)
✓ Data scaled successfully!
  X_train_scaled shape: (286, 12)
  X_test_scaled shape: (72, 12)


## 6. Model 1: Random Forest

In [12]:
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train_scaled, y_train)
rf_pred = rf_model.predict(X_test_scaled)

rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_mae = mean_absolute_error(y_test, rf_pred)
rf_r2 = r2_score(y_test, rf_pred)

print('Random Forest Results:')
print(f'  RMSE: {rf_rmse:.4f}')
print(f'  MAE: {rf_mae:.4f}')
print(f'  R²: {rf_r2:.4f}')

Random Forest Results:
  RMSE: 3.1813
  MAE: 3.0068
  R²: -3.0260


In [13]:
# Feature Importance
feat_imp = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print('Top 10 Features:')
print(feat_imp.head(10))

Top 10 Features:
                           feature  importance
7         Water_Level_(mMSL)_lag_1    0.492732
2                      day_of_year    0.196514
10  Water_Level_(mMSL)_roll_mean_7    0.122654
9         Water_Level_(mMSL)_lag_7    0.075445
8         Water_Level_(mMSL)_lag_3    0.065232
4                             week    0.037798
11   Water_Level_(mMSL)_roll_std_7    0.008612
3                            month    0.000635
0                     Inflow_(MCM)    0.000208
1                    Rainfall_(mm)    0.000126


## 7. Model 2: Gradient Boosting

In [14]:
gb_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42
)

gb_model.fit(X_train_scaled, y_train)
gb_pred = gb_model.predict(X_test_scaled)

gb_rmse = np.sqrt(mean_squared_error(y_test, gb_pred))
gb_mae = mean_absolute_error(y_test, gb_pred)
gb_r2 = r2_score(y_test, gb_pred)

print('Gradient Boosting Results:')
print(f'  RMSE: {gb_rmse:.4f}')
print(f'  MAE: {gb_mae:.4f}')
print(f'  R²: {gb_r2:.4f}')

Gradient Boosting Results:
  RMSE: 2.8763
  MAE: 2.7027
  R²: -2.2912


## 8. Model 3: LSTM

In [15]:
# Create sequences for LSTM
def create_sequences(X, y, seq_length=14):
    Xs, ys = [], []
    for i in range(len(X) - seq_length):
        Xs.append(X[i:(i + seq_length)])
        ys.append(y[i + seq_length])
    return np.array(Xs), np.array(ys)

SEQ_LEN = 14
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled.flatten(), SEQ_LEN)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled.flatten(), SEQ_LEN)

print(f'LSTM shapes: X_train={X_train_seq.shape}, X_test={X_test_seq.shape}')

LSTM shapes: X_train=(272, 14, 12), X_test=(58, 14, 12)


In [16]:
# Build LSTM
lstm_model = Sequential([
    LSTM(64, activation='relu', return_sequences=True, input_shape=(SEQ_LEN, X_train_seq.shape[2])),
    Dropout(0.2),
    LSTM(32, activation='relu'),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
lstm_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 14, 64)         │        19,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,673 (127.63 KB)

 Trainable params: 32,673 (127.63 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Train LSTM
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = lstm_model.fit(
    X_train_seq, y_train_seq,
    epochs=100,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 7s 172ms/step - loss: 0.5954 - mae: 0.7242 - val_loss: 0.0319 - val_mae: 0.1683
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.1562 - mae: 0.3239 - val_loss: 0.1953 - val_mae: 0.4340
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0808 - mae: 0.2255 - val_loss: 0.0768 - val_mae: 0.2522
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - loss: 0.0735 - mae: 0.2238 - val_loss: 0.0545 - val_mae: 0.2132
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.0530 - mae: 0.1846 - val_loss: 0.0529 - val_mae: 0.2070
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0437 - mae: 0.1639 - val_loss: 0.0206 - val_mae: 0.1300
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0325 - mae: 0.1310 - val_loss: 0.0204 - val_mae: 0.1163
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0289 - mae: 0.1366 - val_loss: 0.0257 - val_mae: 0.1171
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0272 - mae:

In [18]:
# LSTM Evaluation
lstm_pred_scaled = lstm_model.predict(X_test_seq)
lstm_pred = scaler_y.inverse_transform(lstm_pred_scaled)
y_test_actual = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1))

lstm_rmse = np.sqrt(mean_squared_error(y_test_actual, lstm_pred))
lstm_mae = mean_absolute_error(y_test_actual, lstm_pred)
lstm_r2 = r2_score(y_test_actual, lstm_pred)

print('LSTM Results:')
print(f'  RMSE: {lstm_rmse:.4f}')
print(f'  MAE: {lstm_mae:.4f}')
print(f'  R²: {lstm_r2:.4f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 367ms/step
LSTM Results:
  RMSE: 2.1945
  MAE: 2.1028
  R²: -5.6267


## 9. Probabilistic Forecasting

In [19]:
# Quantile Regression for uncertainty estimation
quantiles = [0.1, 0.5, 0.9]
q_models = {}
q_preds = {}

for q in quantiles:
    model = GradientBoostingRegressor(
        loss='quantile', alpha=q,
        n_estimators=100, max_depth=5, random_state=42
    )
    model.fit(X_train_scaled, y_train)
    q_models[q] = model
    q_preds[q] = model.predict(X_test_scaled)
    print(f'Quantile {q} trained')

# Coverage check
coverage = ((y_test.values >= q_preds[0.1]) & (y_test.values <= q_preds[0.9])).mean()
print(f'\n80% Interval Coverage: {coverage:.2%}')

Quantile 0.1 trained
Quantile 0.5 trained
Quantile 0.9 trained

80% Interval Coverage: 8.33%


## 10. Model Comparison

In [20]:
results = pd.DataFrame({
    'Model': ['Random Forest', 'Gradient Boosting', 'LSTM'],
    'RMSE': [rf_rmse, gb_rmse, lstm_rmse],
    'MAE': [rf_mae, gb_mae, lstm_mae],
    'R²': [rf_r2, gb_r2, lstm_r2]
}).sort_values('RMSE')

print('='*50)
print('MODEL COMPARISON')
print('='*50)
print(results.to_string(index=False))
print(f'\n🏆 Best: {results.iloc[0]["Model"]}')

MODEL COMPARISON
            Model     RMSE      MAE        R²
             LSTM 2.194475 2.102814 -5.626746
Gradient Boosting 2.876325 2.702729 -2.291157
    Random Forest 3.181277 3.006846 -3.026018

🏆 Best: LSTM


## 11. Save Models

In [21]:
import joblib
import os

os.makedirs('models', exist_ok=True)

# Save models
joblib.dump(rf_model, 'models/random_forest.pkl')
joblib.dump(gb_model, 'models/gradient_boosting.pkl')
joblib.dump(scaler_X, 'models/scaler_X.pkl')
joblib.dump(scaler_y, 'models/scaler_y.pkl')
for q, m in q_models.items():
    joblib.dump(m, f'models/quantile_{int(q*100)}.pkl')
lstm_model.save('models/lstm_model.keras')

# Save results
results.to_csv('model_comparison.csv', index=False)

print('All models saved to ./models/')

All models saved to ./models/
